<div align="center">

# NYU Agentic AI Workshop - Session 2

## Advanced MCP Features and Agentic AI Survey

### Part 1: Making CRUD tools to give agent all the control it needs

</div>


---

<br/>

<div align="center">

# Our Project: The Personalized Newspaper Agent 📰

## The Vision

**"I want a personalized newspaper delivered to me for my 30-minute train commute."**

-> Aggregates news from multiple sources

-> Filters by my interests

-> Creates a beautiful, readable format

-> Emails it to me

Let's build this step by step!


### Checking articles on: https://mail.google.com/mail/u/0/#search/%22Generated+by+Newspaper+Creation+Agent%22

</div>


<div align="center">

<img src="https://github.com/adityaarunsinghal/agentic-ai-workshop-2025/blob/main/notebooks/media/mcp_restaurant_00_analogy.png?raw=true" width="800" alt="mcp_restaurant_00_analogy">

</div>

In [1]:
# Setup and Imports
import os
import sys
from datetime import datetime
from pathlib import Path
from typing import Dict, List

import nest_asyncio
from fastmcp import Context, FastMCP
from dotenv import load_dotenv

# Load .env from project root
env_path = Path.cwd().parent / ".env"
load_dotenv(env_path)

sys.path.append("..")

print(f"✅ Loaded .env from: {env_path}")

# Add server to path
sys.path.insert(0, str(Path.cwd().parent / "src" / "server"))

nest_asyncio.apply()

# Service imports
from config.settings import get_settings
from services.article_memory_v1 import ArticleMemoryService
from services.email_service import EmailService
from services.http_client import HackerNewsClient, fetch_content
from services.interests_file import InterestsFileService
from services.newspaper_service import NewspaperService

print(f"📁 Working directory: {Path.cwd()}")
print("✅ All imports successful!")

✅ Loaded .env from: /Users/adi/Documents/GitHub/agentic-ai-workshop-2025/.env
📁 Working directory: /Users/adi/Documents/GitHub/agentic-ai-workshop-2025/notebooks
✅ All imports successful!


<div align="center">

<img src="https://github.com/adityaarunsinghal/agentic-ai-workshop-2025/blob/main/notebooks/media/mcp_restaurant_12_transport.png?raw=true" width="500" alt="mcp_restaurant_12_transport">
<!-- <img src="https://github.com/adityaarunsinghal/agentic-ai-workshop-2025/blob/main/notebooks/media/mcp_restaurant_12_transport.png?raw=true" width="500" alt="mcp_restaurant_12_transport"> -->

### Nest AsyncIO needed because Jupyter has its own server. There are multiple ways to "serve" MCP

</div>

In [2]:
# Application Context Setup
from contextlib import asynccontextmanager
from dataclasses import dataclass
from typing import Callable
from functools import wraps
import warnings


@dataclass
class AppContext:
    """Application context with all services."""

    hn_client: HackerNewsClient
    newspaper_service: NewspaperService
    email_service: EmailService
    settings: object


MCP_INSTRUCTIONS_V1 = f"""This server contains all tools needed for a newspaper editor with sophisticated capabilities.

CONTENT DISCOVERY:
- Fetch stories from Hacker News (top, new, best, ask, show, job)
- Fetch full article content from any URL
- [Not included in this server] Search the web using Brave
- [Not included in this server] Deep research using Perplexity

NEWSPAPER CREATION:
You have a comprehensive editorial suite with tools for:
- Structure: Create sections, manage layouts, reorder
- Articles: Add content with rich formatting options
- Editorial: Add notes, theme highlights, statistics
- Polish: Previews, validation, table of contents
- Delivery: Generate HTML and send via email

EDITORIAL JUDGMENT:
- Decide appropriate depth and tone for each story
- Connect related stories across sources
- Identify emerging themes and trends
- Adapt presentation to reading time available

INTELLIGENT FEATURES:
- Validate newspapers before sending
- Store finished newspapers for context

This server enables insightful, well-researched, beautifully formatted newspapers."""


EMAIL_SERVICE = EmailService(
    {
        "server": "smtp.gmail.com",
        "port": 465,
        "use_tls": False,
        "use_ssl": True,
        "username": os.getenv(
            "MCP_SMTP_FROM_EMAIL",
        ),
        "password": os.getenv("MCP_SMTP_PASSWORD", ""),
        "from_email": os.getenv(
            "MCP_SMTP_FROM_EMAIL",
        ),
        "from_name": "Newspaper Creator Agent",
    }
)


@asynccontextmanager
async def app_lifespan(mcp: FastMCP):
    """Initialize all services for the newspaper agent."""
    print("🚀 Starting Advanced Newspaper Agent MCP Server")

    # Get settings
    settings = get_settings()
    settings.data_dir.mkdir(parents=True, exist_ok=True)

    # Initialize services
    hn_client = HackerNewsClient()
    newspaper_service = NewspaperService(settings.data_dir)

    # Email service
    email_service = EMAIL_SERVICE

    print("✅ All services initialized!")

    try:
        yield AppContext(
            hn_client=hn_client,
            newspaper_service=newspaper_service,
            email_service=email_service,
            settings=settings,
        )
    finally:
        print("👋 Shutting down MCP Server")


# Tool Registry - allows tools to survive MCP re-instantiation
TOOL_REGISTRY: List[Callable] = []


def register_tool(func: Callable) -> Callable:
    """Register a tool so it can be reapplied to new MCP instances."""
    TOOL_REGISTRY.append(func)
    return func


def apply_registered_tools(mcp_instance: FastMCP):
    """Apply all registered tools to an MCP instance."""
    for tool_func in TOOL_REGISTRY:
        mcp_instance.tool()(tool_func)
    print(f"✅ Applied {len(TOOL_REGISTRY)} registered tools to MCP instance")


print("✅ Application context manager ready!")

✅ Application context manager ready!


In [3]:
mcp = FastMCP(
    name="newspaper-creation-agent",
    instructions=MCP_INSTRUCTIONS_V1,
    lifespan=app_lifespan,
)

print("✅ MCP Server created!")

✅ MCP Server created!


In [4]:
@register_tool
async def fetch_hn_stories(
    count: int = 20, category: str = "top", ctx: Context = None
) -> str:
    """
    Fetch stories from Hacker News.

    Args:
        count: Number of stories (1-30)
        category: Category (top, new, best, ask, show, job)

    Returns:
        JSON list of stories with id, title, url, score, by
    """
    if not 1 <= count <= 30:
        return "❌ Count must be between 1 and 30"

    valid_categories = ["top", "new", "best", "ask", "show", "job"]
    if category not in valid_categories:
        return f"❌ Invalid category. Must be one of: {', '.join(valid_categories)}"

    client = ctx.request_context.lifespan_context.hn_client

    # Map to API endpoints
    endpoint_map = {
        "top": "topstories",
        "new": "newstories",
        "best": "beststories",
        "ask": "askstories",
        "show": "showstories",
        "job": "jobstories",
    }

    try:
        story_ids = await client.get_story_ids(endpoint_map[category], count)

        stories = []
        for story_id in story_ids:
            story = await client.get_item(story_id)
            if story and story.get("title"):
                stories.append(
                    {
                        "id": story_id,
                        "title": story["title"],
                        "url": story.get(
                            "url", f"https://news.ycombinator.com/item?id={story_id}"
                        ),
                        "score": story.get("score", 0),
                        "by": story.get("by", "unknown"),
                        "time": story.get("time", 0),
                    }
                )

        result = f"# 📰 {len(stories)} Hacker News Stories ({category})\n\n"
        for i, story in enumerate(stories, 1):
            result += f"{i}. **{story['title']}** ({story['score']} points by {story['by']})\n"
            result += f"   {story['url']}\n\n"

        return result

    except Exception as e:
        return f"❌ Failed to fetch stories: {e}"


@register_tool
async def fetch_article_content(url: str, ctx: Context = None) -> str:
    """
    Fetch and convert article content to clean markdown.

    Args:
        url: Article URL

    Returns:
        Markdown-formatted article content
    """
    try:
        content = await fetch_content(url, max_length=32000)
        return f"# Content from {url}\n\n{content}"
    except Exception as e:
        return f"❌ Failed to fetch content from {url}: {e}"


# Note: search_web and research_topic are provided by brave and perplexity MCP servers

In [5]:
# ============= NEWSPAPER CREATION: ARTICLE TOOLS =============


@register_tool
async def add_article(
    newspaper_id: str,
    section_title: str,
    title: str,
    content: str,
    url: str = "",
    author: str = "",
    source: str = "",
    placement: str = "standard",
    tags: List[str] = None,
    ctx: Context = None,
) -> str:
    """
    Add an article to a newspaper section.

    Args:
        newspaper_id: Newspaper ID
        section_title: Section to add to
        title: Article title
        content: Article content or summary
        url: Source URL
        author: Article author
        source: Content source (hn, web, etc.)
        placement: Placement (lead, standard, sidebar, quick-read)
        tags: Topic tags

    Returns:
        Confirmation with reading time
    """
    newspaper_service = ctx.request_context.lifespan_context.newspaper_service

    article_data = {
        "title": title,
        "content": content,
        "url": url,
        "author": author,
        "source": source,
        "tags": tags or [],
    }

    result = newspaper_service.add_article(
        newspaper_id, section_title, article_data, placement
    )

    if result["success"]:
        return f"""✅ Added article to {result['section']}

**Title:** {result['article_title']}
**Reading Time:** {result['reading_time']} min
**Placement:** {placement}"""
    else:
        return f"❌ {result['error']}"


@register_tool
async def set_article_format(
    newspaper_id: str,
    section_title: str,
    article_title: str,
    show_image: bool = False,
    image_url: str = "",
    image_caption: str = "",
    pull_quote: str = "",
    key_points: List[str] = None,
    callout_box: str = "",
    ctx: Context = None,
) -> str:
    """
    Apply rich formatting to an article.

    Args:
        newspaper_id: Newspaper ID
        section_title: Section title
        article_title: Article title
        show_image: Whether to show image
        image_url: Image URL
        image_caption: Image caption
        pull_quote: Highlighted quote from article
        key_points: List of key takeaways
        callout_box: Special note/context box

    Returns:
        Confirmation
    """
    newspaper_service = ctx.request_context.lifespan_context.newspaper_service

    format_options = {}
    if show_image:
        format_options["show_image"] = True
    if image_url:
        format_options["image_url"] = image_url
    if image_caption:
        format_options["image_caption"] = image_caption
    if pull_quote:
        format_options["pull_quote"] = pull_quote
    if key_points:
        format_options["key_points"] = key_points
    if callout_box:
        format_options["callout_box"] = callout_box

    result = newspaper_service.set_article_format(
        newspaper_id, section_title, article_title, format_options
    )

    if result["success"]:
        return f"✅ Applied formatting to '{result['article_title']}'"
    else:
        return f"❌ {result['error']}"


@register_tool
async def highlight_article(
    newspaper_id: str,
    section_title: str,
    article_title: str,
    highlight_type: str,
    ctx: Context = None,
) -> str:
    """
    Add a highlight badge to an article.

    Args:
        newspaper_id: Newspaper ID
        section_title: Section title
        article_title: Article title
        highlight_type: Type (breaking, trending, exclusive, deep-dive)

    Returns:
        Confirmation
    """
    newspaper_service = ctx.request_context.lifespan_context.newspaper_service

    result = newspaper_service.highlight_article(
        newspaper_id, section_title, article_title, highlight_type
    )

    if result["success"]:
        return f"✅ Added [{highlight_type.upper()}] badge to article"
    else:
        return f"❌ {result['error']}"


@register_tool
async def link_related_articles(
    newspaper_id: str,
    article_title: str,
    related_titles: List[str],
    ctx: Context = None,
) -> str:
    """
    Create "Related Stories" links within the newspaper.

    Args:
        newspaper_id: Newspaper ID
        article_title: Main article title
        related_titles: List of related article titles

    Returns:
        Confirmation
    """
    newspaper_service = ctx.request_context.lifespan_context.newspaper_service

    result = newspaper_service.link_related_articles(
        newspaper_id, article_title, related_titles
    )

    if result["success"]:
        return f"✅ Linked {result['related_count']} related articles to '{result['article_title']}'"
    else:
        return f"❌ {result['error']}"

In [6]:
# ============= NEWSPAPER CREATION: STRUCTURE TOOLS =============


@register_tool
async def create_newspaper_draft(
    title: str, subtitle: str = "", edition_type: str = "standard", ctx: Context = None
) -> str:
    """
    Create a new newspaper draft.

    Args:
        title: Newspaper title
        subtitle: Optional subtitle
        edition_type: Edition type (morning_brief, deep_dive, breaking, weekly, special, standard)

    Returns:
        Newspaper ID and confirmation
    """
    newspaper_service = ctx.request_context.lifespan_context.newspaper_service

    result = newspaper_service.create_draft(title, subtitle, edition_type)

    if result["success"]:
        return f"""✅ Created newspaper draft!

**ID:** {result['newspaper_id']}
**Title:** {result['title']}
**Edition:** {result['edition_type']}

Suggested Next steps:
1. add_section() to create sections
2. add_article() to add content
3. Use formatting tools to polish
4. preview_newspaper() to review
5. send_newspaper() to deliver"""
    else:
        return f"❌ Failed to create newspaper: {result.get('error', 'Unknown error')}"


@register_tool
async def add_section(
    newspaper_id: str,
    section_title: str,
    layout: str = "grid",
    position: int = None,
    ctx: Context = None,
) -> str:
    """
    Add a section to the newspaper.

    Args:
        newspaper_id: Newspaper ID
        section_title: Section title (e.g., "Breaking News", "Deep Dive")
        layout: Layout type (grid, single-column, featured, timeline)
        position: Optional position (None = append to end)

    Returns:
        Confirmation
    """
    newspaper_service = ctx.request_context.lifespan_context.newspaper_service

    result = newspaper_service.add_section(
        newspaper_id, section_title, layout, position
    )

    if result["success"]:
        return f"✅ Added section '{result['section_title']}' with {result['layout']} layout"
    else:
        return f"❌ {result['error']}"


@register_tool
async def reorder_sections(
    newspaper_id: str, section_order: List[str], ctx: Context = None
) -> str:
    """
    Reorder sections in the newspaper.

    Args:
        newspaper_id: Newspaper ID
        section_order: List of section titles in desired order

    Returns:
        Confirmation
    """
    newspaper_service = ctx.request_context.lifespan_context.newspaper_service

    result = newspaper_service.reorder_sections(newspaper_id, section_order)

    if result["success"]:
        return f"✅ Reordered sections: {' → '.join(result['section_order'])}"
    else:
        return f"❌ {result['error']}"


@register_tool
async def set_section_layout(
    newspaper_id: str, section_title: str, layout: str, ctx: Context = None
) -> str:
    """
    Change the layout of a section.

    Args:
        newspaper_id: Newspaper ID
        section_title: Section title
        layout: New layout (grid, single-column, featured, timeline)

    Returns:
        Confirmation
    """
    newspaper_service = ctx.request_context.lifespan_context.newspaper_service

    result = newspaper_service.set_section_layout(newspaper_id, section_title, layout)

    if result["success"]:
        return f"✅ Changed '{result['section_title']}' layout to {result['layout']}"
    else:
        return f"❌ {result['error']}"


@register_tool
async def remove_section(
    newspaper_id: str, section_title: str, ctx: Context = None
) -> str:
    """
    Remove a section from the newspaper.

    Args:
        newspaper_id: Newspaper ID
        section_title: Section title to remove

    Returns:
        Confirmation
    """
    newspaper_service = ctx.request_context.lifespan_context.newspaper_service

    result = newspaper_service.remove_section(newspaper_id, section_title)

    if result["success"]:
        return f"✅ Removed section '{result['section_title']}'"
    else:
        return f"❌ {result['error']}"

In [7]:
# ============= NEWSPAPER CREATION: PREVIEW & SEND =============


@register_tool
async def preview_newspaper(
    newspaper_id: str, format: str = "markdown", ctx: Context = None
) -> str:
    """
    Preview the newspaper before sending.

    Args:
        newspaper_id: Newspaper ID
        format: Format (markdown, summary)

    Returns:
        Preview of newspaper
    """
    newspaper_service = ctx.request_context.lifespan_context.newspaper_service

    if format == "summary":
        result = newspaper_service.get_stats(newspaper_id)
        if result["success"]:
            stats = result["stats"]
            preview = f"""# 📊 Newspaper Stats

**Title:** {stats['title']}
**Edition:** {stats['edition_type']}
**Articles:** {stats['total_articles']}
**Reading Time:** {stats['total_reading_time']} min
**Sections:** {stats['section_count']}

## Sections:
"""
            for section in stats["sections"]:
                preview += f"- **{section['title']}** ({section['layout']}): {section['article_count']} articles\n"
            return preview
        else:
            return f"❌ {result['error']}"

    else:  # markdown
        result = newspaper_service.preview_markdown(newspaper_id)
        if result["success"]:
            return result["preview"]
        else:
            return f"❌ {result['error']}"


@register_tool
async def get_newspaper_stats(newspaper_id: str, ctx: Context = None) -> str:
    """
    Get detailed statistics about the newspaper.

    Args:
        newspaper_id: Newspaper ID

    Returns:
        Statistics summary
    """
    newspaper_service = ctx.request_context.lifespan_context.newspaper_service

    result = newspaper_service.get_stats(newspaper_id)

    if result["success"]:
        stats = result["stats"]
        output = f"""📊 **Newspaper Statistics**

**Title:** {stats['title']}
**Edition:** {stats['edition_type']}
**Total Articles:** {stats['total_articles']}
**Total Reading Time:** {stats['total_reading_time']} minutes
**Sections:** {stats['section_count']}

**Section Breakdown:**
"""
        for section in stats["sections"]:
            output += f"- {section['title']} ({section['layout']}): {section['article_count']} articles\n"

        return output
    else:
        return f"❌ {result['error']}"


@register_tool
async def validate_newspaper(newspaper_id: str, ctx: Context = None) -> str:
    """
    Validate newspaper for issues before sending.

    Args:
        newspaper_id: Newspaper ID

    Returns:
        Validation report
    """
    newspaper_service = ctx.request_context.lifespan_context.newspaper_service

    result = newspaper_service.validate(newspaper_id)

    if result["success"]:
        if result["valid"]:
            output = "✅ **Newspaper is valid and ready to send!**\n\n"
        else:
            output = "⚠️ **Newspaper has issues:**\n\n"
            for issue in result["issues"]:
                output += f"❌ {issue}\n"
            output += "\n"

        if result["warnings"]:
            output += "**Warnings:**\n"
            for warning in result["warnings"]:
                output += f"⚠️ {warning}\n"

        return output
    else:
        return f"❌ {result['error']}"


@register_tool
async def send_newspaper(
    newspaper_id: str, subject: str = "", ctx: Context = None
) -> str:
    """
    Generate HTML and send newspaper via email.

    Args:
        newspaper_id: Newspaper ID
        subject: Email subject (optional)

    Returns:
        Delivery confirmation
    """
    newspaper_service = ctx.request_context.lifespan_context.newspaper_service
    email_service = ctx.request_context.lifespan_context.email_service
    settings = ctx.request_context.lifespan_context.settings

    # Get newspaper data
    newspaper_data = newspaper_service.get_newspaper_data(newspaper_id)
    if not newspaper_data:
        return f"❌ Newspaper '{newspaper_id}' not found"

    # Generate and save HTML FIRST (before attempting email)
    try:
        html_content = email_service._create_html_version(newspaper_data)
        html_file = settings.data_dir / "newspapers" / f"{newspaper_id}.html"
        with open(html_file, "w", encoding="utf-8") as f:
            f.write(html_content)
        html_saved = True
        html_path = str(html_file)
    except Exception as e:
        html_saved = False
        html_path = None
        print(f"Warning: Failed to save HTML: {e}")

    # Try to send email
    result = email_service.send_newspaper(newspaper_data, subject, version=2)

    if result["success"]:
        return f"✅ **Newspaper sent successfully!**\n\n📰 '{newspaper_data['title']}' delivered to your email.\n📄 HTML saved to: {html_path}"
    else:
        # Email failed, but HTML might be saved
        if html_saved:
            return f"⚠️ **Email failed**: {result.get('error', 'Unknown error')}\n\n✅ However, HTML was saved locally!\n📄 Open this file: {html_path}"
        else:
            return f"❌ Failed to send: {result.get('error', 'Unknown error')}\n❌ HTML generation also failed"


# TODO: Need to add tools for agent to read the FULL newspaper and make text replacement edits

## Checkpoint 1: Basic tools working and MCP inspector

In [8]:
# warnings.filterwarnings("ignore", category=DeprecationWarning)
# apply_registered_tools(mcp)
# os.system("lsof -ti:8080 | xargs kill -9 2>/dev/null")
# await mcp.run_async(transport="streamable-http", port=8080)

<div align="center">

## Checkpoint 1 Results


<img src="https://github.com/adityaarunsinghal/agentic-ai-workshop-2025/blob/main/notebooks/media/session02checkpoint01_short.png?raw=true" width="800" alt="session02checkpoint01_short">
<!-- <img src="media/session02checkpoint01_short.png" width="800" alt="session02checkpoint01_short"> -->

### "make me a newspaper"

_Not enough content_

_I need to instruct it too much every single session to get the results i want_

<img src="https://github.com/adityaarunsinghal/agentic-ai-workshop-2025/blob/main/notebooks/media/session02checkpoint02_token_usage.png?raw=true" width="800" alt="session02checkpoint02_token_usage">
<!-- <img src="media/session02checkpoint02_token_usage.png" width="800" alt="session02checkpoint02_token_usage"> -->
<br/>
<img src="https://github.com/adityaarunsinghal/agentic-ai-workshop-2025/blob/main/notebooks/media/session02checkpoint02_after_prompt.png?raw=true" width="800" alt="session02checkpoint02_after_prompt">
<!-- <img src="media/session02checkpoint02_after_prompt.png" width="800" alt="session02checkpoint02_after_prompt"> -->

### "make me a newspaper but do NOT stop until i have at least 10 mins of reading content and very nicely polished newspaper with diverse sections"


_Good polish_

_Irrelevant to interests_

_Agent can't reference the old newspapers i read_

</div>

In [9]:
@dataclass
class AppContext:
    """Application context with all services."""

    hn_client: HackerNewsClient
    interests_service: InterestsFileService
    article_memory: ArticleMemoryService
    newspaper_service: NewspaperService
    email_service: EmailService
    settings: object


@asynccontextmanager
async def app_lifespan(mcp: FastMCP):
    """Initialize all services for the newspaper agent."""
    print("🚀 Starting Advanced Newspaper Agent MCP Server")

    # Get settings
    settings = get_settings()
    settings.data_dir.mkdir(parents=True, exist_ok=True)

    # Initialize services
    hn_client = HackerNewsClient()
    interests_service = InterestsFileService(settings.data_dir)

    # Initialize memory service
    article_memory = ArticleMemoryService()
    article_memory.initialize(settings.data_dir / "chromadb")

    newspaper_service = NewspaperService(settings.data_dir)

    # Email service
    email_service = EMAIL_SERVICE

    print("✅ All services initialized!")
    print(f"📧 Email: {email_service.server}:{email_service.port}")
    print(f"📁 Data: {settings.data_dir}")
    print(f"🗄️ ChromaDB: {settings.data_dir / 'chromadb'}")

    try:
        yield AppContext(
            hn_client=hn_client,
            interests_service=interests_service,
            article_memory=article_memory,
            newspaper_service=newspaper_service,
            email_service=email_service,
            settings=settings,
        )
    finally:
        print("👋 Shutting down MCP Server")


MCP_INSTRUCTIONS_V2 = f"""This server contains all tools needed for a newspaper editor with sophisticated capabilities.

CONTENT DISCOVERY:
- Fetch stories from Hacker News (top, new, best, ask, show, job)
- Fetch full article content from any URL
- [Not included in this server] Search the web using Brave
- [Not included in this server] Deep research using Perplexity

MEMORY & CONTEXT:
- Access user interests from interests.md file
- Search your archive of past articles (semantic search)
- Review past newspapers you've created
- Store articles for future reference

NEWSPAPER CREATION:
You have a comprehensive editorial suite with tools for:
- Structure: Create sections, manage layouts, reorder
- Articles: Add content with rich formatting options
- Editorial: Add notes, theme highlights, statistics
- Polish: Previews, validation, table of contents
- Delivery: Generate HTML and send via email

EDITORIAL JUDGMENT:
- Prioritize stories based on user interests
- Decide appropriate depth and tone for each story
- Connect related stories across sources
- Identify emerging themes and trends
- Adapt presentation to reading time available

INTELLIGENT FEATURES:
- Use sampling for summarization and analysis
- Extract themes from multiple articles
- Generate editorials and synthesis pieces
- Validate newspapers before sending
- Store finished newspapers for context

Current date: {datetime.now().strftime('%A, %B %d, %Y')}

This server enables insightful, well-researched, beautifully formatted newspapers."""

mcp = FastMCP(
    name="advanced-newspaper-agent",
    instructions=MCP_INSTRUCTIONS_V2,
    lifespan=app_lifespan,
)

In [10]:
# ============= INTEREST MANAGEMENT TOOLS =============


@register_tool
async def read_interests(ctx: Context = None) -> str:
    """
    Read current user interests from interests.md file.

    Returns:
        Formatted interests summary
    """
    interests_service = ctx.request_context.lifespan_context.interests_service
    interests = interests_service.read_interests()

    if "error" in interests:
        return f"❌ Failed to read interests: {interests['error']}"

    result = "# 📋 Your Interests\n\n"
    result += f"**Last Updated:** {interests['last_updated']}\n\n"

    if interests["topics"]:
        result += f"**Topics ({len(interests['topics'])}):**\n"
        for topic in interests["topics"]:
            result += f"- {topic}\n"
        result += "\n"

    if interests["sources"]:
        result += "**Preferred Sources:**\n"
        for source in interests["sources"]:
            result += f"- {source}\n"
        result += "\n"

    result += f"**Summary Style:** {interests['style']}\n\n"

    if interests["notes"]:
        result += "**Notes:**\n"
        for note in interests["notes"]:
            result += f"- {note}\n"

    return result


@register_tool
async def add_interests(topics: List[str], ctx: Context = None) -> str:
    """
    Add topics to your interests.

    Args:
        topics: List of topics to add

    Returns:
        Confirmation with updated count
    """
    interests_service = ctx.request_context.lifespan_context.interests_service
    result = interests_service.add_topics(topics)

    if result["success"]:
        output = "✅ **Added interests!**\n\n"
        if result["added"]:
            output += "**New topics:**\n"
            for topic in result["added"]:
                output += f"- {topic}\n"
        else:
            output += "*All topics were already in your interests*\n"
        output += f"\n**Total topics:** {result['total_topics']}"
        return output
    else:
        return f"❌ {result['error']}"


@register_tool
async def remove_interests(topics: List[str], ctx: Context = None) -> str:
    """
    Remove topics from your interests.

    Args:
        topics: List of topics to remove

    Returns:
        Confirmation with remaining count
    """
    interests_service = ctx.request_context.lifespan_context.interests_service
    result = interests_service.remove_topics(topics)

    if result["success"]:
        output = "✅ **Removed interests!**\n\n"
        if result["removed"]:
            output += "**Removed topics:**\n"
            for topic in result["removed"]:
                output += f"- {topic}\n"
        else:
            output += "*No topics were found to remove*\n"
        output += f"\n**Remaining topics:** {result['remaining_topics']}"
        return output
    else:
        return f"❌ {result['error']}"

<div align="center">

<img src="https://github.com/adityaarunsinghal/agentic-ai-workshop-2025/blob/main/notebooks/media/mcp_restaurant_06_prompts.png?raw=true" width="500" alt="mcp_restaurant_06_prompts">
<!-- <img src="https://github.com/adityaarunsinghal/agentic-ai-workshop-2025/blob/main/notebooks/media/mcp_restaurant_06_prompts.png?raw=true" width="500" alt="mcp_restaurant_06_prompts"> -->

### We know our MCP Servers better than end users do

</div>

In [11]:
MORNING_BRIEF_PROMPT_v1 = """Create a morning newspaper that's perfect for my commute (under 20 min read).

Start by understanding what I care about - check my interests file to see what topics resonate with me. Pull from Hacker News top stories, but be selective - only include what truly matters to me or represents important developments.

Structure it as a "Morning Tech Brief" with a featured breaking news section for 2-3 deeply covered stories, plus a grid of quick reads for another 4-5 interesting items. The breaking stories should have full context and rich summaries, while quick reads can be more concise.

As you work through articles, store interesting ones in the archive for future reference. Look for common threads - what themes are emerging across these stories today? Capture that insight in an editor's note that helps me see the bigger picture.

Before finalizing, show me what you've built and get my input - would I prefer more depth or broader coverage given my time budget? Adjust accordingly.

When it's polished and I'm ready, send it to my email and archive it for history.

Remember: quality over quantity. Better to have 5-7 perfectly curated stories than 15 mediocre ones. Make editorial judgments based on my interests, not just popularity.
Do NOT STOP until you have at least 10 mins of reading content."""

@mcp.prompt()
async def create_morning_brief() -> str:
    """Workflow for creating a quick morning newspaper."""
    return MORNING_BRIEF_PROMPT_v1

In [12]:
file_path = "../src/server/data/interests.md"
os.makedirs(os.path.dirname(file_path), exist_ok=True)

python_code = """# User Interests
Last Updated: xxx

## Topics
<!-- Add your topics of interest, one per line -->
- Agentic AI only

## Preferred Sources
- Hacker News
- Any links that perplexity gives

## Summary Style
<!-- Your preferred summary style: brief, detailed, technical -->
- 

## Notes
<!-- Any additional notes about your interests -->
- Nothing more for now

"""

with open(file_path, "w") as f:
    f.write(python_code)

print(f"File created successfully at: {file_path}")

File created successfully at: ../src/server/data/interests.md


## Checkpoint 2: Interests captured and MCP Prompts

In [13]:
# warnings.filterwarnings("ignore", category=DeprecationWarning)
# apply_registered_tools(mcp)
# os.system("lsof -ti:8080 | xargs kill -9 2>/dev/null")
# await mcp.run_async(transport="streamable-http", port=8080)


<div align="center">

## Checkpoint 2 Results

</div>



<div align="center">

[📄 Open Newspaper in Browser](https://htmlpreview.github.io/?https://github.com/adityaarunsinghal/agentic-ai-workshop-2025/blob/main/examples/newspaper_20251008_094147.html)


### "Morning Brief Prompt"

_Very high latency_

_LOTS of tool calls_

_Still not contextualized with previous work and being "reborn" each time_

</div>

In [14]:
# ============= MEMORY & STORAGE TOOLS =============


@register_tool
async def store_article(
    url: str,
    content: str,
    title: str = "",
    source: str = "web",
    topics: List[str] = None,
    summary: str = "",
    ctx: Context = None,
) -> str:
    """
    Store article in your archive for future reference.

    Args:
        url: Article URL
        content: Full article content
        title: Article title
        source: Source (hn, web, arxiv, etc.)
        topics: List of topic tags
        summary: Brief summary

    Returns:
        Storage confirmation
    """
    article_memory = ctx.request_context.lifespan_context.article_memory

    result = article_memory.store_article(
        url=url,
        content=content,
        title=title,
        source=source,
        topics=topics or [],
        summary=summary,
    )

    if result["success"]:
        return f"✅ Stored article: '{result['title']}'\n📍 ID: {result['doc_id']}"
    else:
        return f"❌ Failed to store article: {result['error']}"


@register_tool
async def search_article_archive(
    query: str,
    limit: int = 5,
    source_filter: str = "",
    topic_filter: str = "",
    ctx: Context = None,
) -> str:
    """
    Search your article archive using semantic similarity.

    Args:
        query: Search query
        limit: Maximum results (1-20)
        source_filter: Filter by source (optional)
        topic_filter: Filter by topic (optional)

    Returns:
        List of relevant articles with similarity scores
    """
    if not 1 <= limit <= 20:
        return "❌ Limit must be between 1 and 20"

    article_memory = ctx.request_context.lifespan_context.article_memory

    articles = article_memory.search_articles(
        query=query,
        limit=limit,
        source_filter=source_filter if source_filter else None,
        topic_filter=topic_filter if topic_filter else None,
    )

    if not articles:
        result = f"# 🔍 No articles found for '{query}'\n\n"
        result += "Try:\n- Broader search terms\n- Removing filters\n- Storing more articles first"
        return result

    result = f"# 🔍 Found {len(articles)} articles for '{query}'\n\n"

    for i, article in enumerate(articles, 1):
        result += f"## {i}. {article['title']}\n"
        result += f"**Similarity:** {article['similarity']:.1%} | "
        result += f"**Source:** {article['source']} | "
        result += f"**Words:** {article['word_count']:,}\n"
        if article["topics"]:
            result += f"**Topics:** {', '.join(article['topics'])}\n"
        result += f"**URL:** {article['url']}\n\n"
        if article["summary"]:
            result += f"{article['summary']}\n\n"
        result += f"*Preview:* {article['content_preview']}\n\n"
        result += "---\n\n"

    return result


@register_tool
async def store_newspaper(newspaper_id: str, ctx: Context = None) -> str:
    """
    Store completed newspaper in archive for history.

    Args:
        newspaper_id: Newspaper ID to store

    Returns:
        Storage confirmation
    """
    newspaper_service = ctx.request_context.lifespan_context.newspaper_service
    article_memory = ctx.request_context.lifespan_context.article_memory

    # Get newspaper data
    newspaper_data = newspaper_service.get_newspaper_data(newspaper_id)
    if not newspaper_data:
        return f"❌ Newspaper '{newspaper_id}' not found"

    # Store in memory
    result = article_memory.store_newspaper(newspaper_id, newspaper_data)

    if result["success"]:
        return f"✅ Stored newspaper '{newspaper_data['title']}' in archive\n📅 {result['timestamp'][:10]}"
    else:
        return f"❌ Failed to store newspaper: {result['error']}"


@register_tool
async def search_past_newspapers(
    days_back: int = 7, query: str = "", ctx: Context = None
) -> str:
    """
    Search or list past newspapers from your archive.

    Args:
        days_back: How many days back to search (1-30)
        query: Optional semantic search query

    Returns:
        List of past newspapers with metadata
    """
    if not 1 <= days_back <= 30:
        return "❌ days_back must be between 1 and 30"

    article_memory = ctx.request_context.lifespan_context.article_memory

    newspapers = article_memory.search_newspapers(
        days_back=days_back, query=query if query else None
    )

    if not newspapers:
        return f"# 📰 No newspapers found in the past {days_back} days"

    result = f"# 📰 Found {len(newspapers)} newspapers (past {days_back} days)\n\n"

    for i, paper in enumerate(newspapers, 1):
        result += f"## {i}. {paper['title']}\n"
        result += f"**ID:** {paper['newspaper_id']}\n"
        result += f"**Date:** {paper['timestamp'][:10]}\n"
        result += f"**Type:** {paper['edition_type']} | "
        result += f"**Articles:** {paper['article_count']} | "
        result += f"**Reading Time:** {paper['reading_time']} min\n"
        if paper["topics"]:
            result += f"**Topics:** {', '.join(paper['topics'])}\n"
        if paper["tone"]:
            result += f"**Tone:** {paper['tone']}\n"
        result += "\n---\n\n"

    return result

In [15]:
# ============= NEWSPAPER CREATION: EDITORIAL TOOLS =============


@register_tool
async def add_editors_note(
    newspaper_id: str,
    content: str,
    placement: str = "top",
    style: str = "standard",
    ctx: Context = None,
) -> str:
    """
    Add an editor's note to the newspaper.

    Args:
        newspaper_id: Newspaper ID
        content: Note content
        placement: Placement (top, bottom, section:<name>)
        style: Style (standard, highlighted, sidebar)

    Returns:
        Confirmation
    """
    newspaper_service = ctx.request_context.lifespan_context.newspaper_service

    result = newspaper_service.add_editors_note(newspaper_id, content, placement, style)

    if result["success"]:
        return f"✅ Added editor's note ({placement}, {style} style)"
    else:
        return f"❌ {result['error']}"


@register_tool
async def add_theme_highlight(
    newspaper_id: str,
    theme: str,
    description: str,
    related_articles: List[str],
    ctx: Context = None,
) -> str:
    """
    Add a theme/trend box connecting multiple articles.

    Example: "3 stories today touch on AI governance - here's why it matters"

    Args:
        newspaper_id: Newspaper ID
        theme: Theme name
        description: Theme description
        related_articles: List of related article titles

    Returns:
        Confirmation
    """
    newspaper_service = ctx.request_context.lifespan_context.newspaper_service

    result = newspaper_service.add_theme_highlight(
        newspaper_id, theme, description, related_articles
    )

    if result["success"]:
        return f"✅ Added theme highlight: '{result['theme']}' connecting {len(related_articles)} articles"
    else:
        return f"❌ {result['error']}"


@register_tool
async def add_stats_callout(
    newspaper_id: str,
    section_title: str,
    stats: List[Dict],
    position: str = "top",
    ctx: Context = None,
) -> str:
    """
    Add a statistics callout box to a section.

    Example: "1.2M GitHub stars | 450 HN points | Released 2 days ago"

    Args:
        newspaper_id: Newspaper ID
        section_title: Section title
        stats: List of {label, value, context} dicts
        position: Position (top, bottom, sidebar)

    Returns:
        Confirmation
    """
    newspaper_service = ctx.request_context.lifespan_context.newspaper_service

    result = newspaper_service.add_stats_callout(
        newspaper_id, section_title, stats, position
    )

    if result["success"]:
        return f"✅ Added stats callout to '{result['section']}' with {len(stats)} statistics"
    else:
        return f"❌ {result['error']}"


@register_tool
async def add_resource_box(
    newspaper_id: str,
    section_title: str,
    title: str,
    resources: List[Dict],
    ctx: Context = None,
) -> str:
    """
    Add a "Further Reading" or "Resources" box.

    Args:
        newspaper_id: Newspaper ID
        section_title: Section title
        title: Box title
        resources: List of {title, url, description} dicts

    Returns:
        Confirmation
    """
    newspaper_service = ctx.request_context.lifespan_context.newspaper_service

    result = newspaper_service.add_resource_box(
        newspaper_id, section_title, title, resources
    )

    if result["success"]:
        return f"✅ Added resource box '{title}' to '{result['section']}' with {len(resources)} resources"
    else:
        return f"❌ {result['error']}"

In [16]:
# ============= NEWSPAPER CREATION: METADATA & POLISH =============


@register_tool
async def set_newspaper_metadata(
    newspaper_id: str,
    tone: str = "",
    target_audience: str = "",
    topics: List[str] = None,
    ctx: Context = None,
) -> str:
    """
    Set newspaper-level metadata.

    Args:
        newspaper_id: Newspaper ID
        tone: Editorial tone (analytical, educational, skeptical, enthusiastic)
        target_audience: Target audience description
        topics: List of main topics covered

    Returns:
        Confirmation
    """
    newspaper_service = ctx.request_context.lifespan_context.newspaper_service

    metadata = {}
    if tone:
        metadata["tone"] = tone
    if target_audience:
        metadata["target_audience"] = target_audience
    if topics:
        metadata["topics"] = topics

    result = newspaper_service.set_metadata(newspaper_id, metadata)

    if result["success"]:
        return "✅ Updated newspaper metadata"
    else:
        return f"❌ {result['error']}"


@register_tool
async def calculate_reading_times(newspaper_id: str, ctx: Context = None) -> str:
    """
    Recalculate reading time estimates for all articles.

    Args:
        newspaper_id: Newspaper ID

    Returns:
        Total reading time
    """
    newspaper_service = ctx.request_context.lifespan_context.newspaper_service

    result = newspaper_service.calculate_reading_times(newspaper_id)

    if result["success"]:
        return f"✅ Total reading time: {result['total_reading_time']} minutes"
    else:
        return f"❌ {result['error']}"


@register_tool
async def add_table_of_contents(
    newspaper_id: str, style: str = "compact", ctx: Context = None
) -> str:
    """
    Add a table of contents to the newspaper.

    Args:
        newspaper_id: Newspaper ID
        style: TOC style (compact, detailed, visual)

    Returns:
        Confirmation
    """
    newspaper_service = ctx.request_context.lifespan_context.newspaper_service

    result = newspaper_service.add_table_of_contents(newspaper_id, style)

    if result["success"]:
        return f"✅ Added {result['toc_style']} table of contents"
    else:
        return f"❌ {result['error']}"

In [17]:
MORNING_BRIEF_PROMPT_v2 = MORNING_BRIEF_PROMPT_v1 + """\nPlease also search the article storage to draw connections to what we have discovered together on previous mornings.
The newspaper must contain references to old newspapers and newspaper contents. You may also update my interests."""

@mcp.prompt()
async def create_morning_brief() -> str:
    """Workflow for creating a quick morning newspaper."""
    return MORNING_BRIEF_PROMPT_v2

[10/08/25 16:32:55] WARNING  Prompt already exists: create_morning_brief                      ]8;id=267941;file:///Users/adi/Documents/GitHub/agentic-ai-workshop-2025/.venv/lib/python3.13/site-packages/fastmcp/prompts/prompt_manager.py\prompt_manager.py]8;;\:]8;id=150449;file:///Users/adi/Documents/GitHub/agentic-ai-workshop-2025/.venv/lib/python3.13/site-packages/fastmcp/prompts/prompt_manager.py#142\142]8;;\

<div align="center">

<img src="https://github.com/adityaarunsinghal/agentic-ai-workshop-2025/blob/main/notebooks/media/mcp_restaurant_05_resources.png?raw=true" width="500" alt="mcp_restaurant_05_resources">

### No tool calls needed to check the ingredients/nutritional facts

</div>

In [18]:
@mcp.resource("file://interests.md")
async def get_interests_resource(ctx: Context = None) -> str:
    """User's interests file - topics, sources, preferences."""
    interests_service = ctx.request_context.lifespan_context.interests_service
    interests = interests_service.read_interests()

    content = "# User Interests\n\n"
    content += f"**Last Updated:** {interests.get('last_updated', 'Unknown')}\n\n"

    if interests.get("topics"):
        content += "## Topics\n"
        for topic in interests["topics"]:
            content += f"- {topic}\n"
        content += "\n"

    if interests.get("sources"):
        content += "## Preferred Sources\n"
        for source in interests["sources"]:
            content += f"- {source}\n"
        content += "\n"

    content += f"## Summary Style\n- {interests.get('style', 'detailed')}\n\n"

    if interests.get("notes"):
        content += "## Notes\n"
        for note in interests["notes"]:
            content += f"- {note}\n"

    return content

## Checkpoint 3: Way more tools for better polish control and Agent can search its own historical work

In [19]:
# warnings.filterwarnings("ignore", category=DeprecationWarning)
# apply_registered_tools(mcp)
# os.system("lsof -ti:8080 | xargs kill -9 2>/dev/null")
# await mcp.run_async(transport="streamable-http", port=8080)

<div align="center">

## Checkpoint 3 Results

<img src="https://github.com/adityaarunsinghal/agentic-ai-workshop-2025/blob/main/notebooks/media/session02checkpoint03_remembers.png?raw=true" width="1500" alt="session02checkpoint03_remembers">
<!-- <img src="media/session02checkpoint03_remembers.png" width="1500" alt="session02checkpoint03_remembers"> -->

### "Morning Brief Prompt v2"

_Much more contextual, agent can empower itself run over run_

_Added too many interests ??_

_Easy to have 2 modes: Discovery and Rudimentary Personalized_

_User has more visibility over shared data without needing to call tools_


<img src="https://github.com/adityaarunsinghal/agentic-ai-workshop-2025/blob/main/notebooks/media/session02checkpoint03_interests_added.png?raw=true" width="1500" alt="session02checkpoint03_interests_added">
<!-- <img src="media/session02checkpoint03_interests_added.png" width="1500" alt="session02checkpoint03_interests_added"> -->

_Added too many interests ??_


</div>

In [20]:
from dataclasses import dataclass


@dataclass
class Confirmation:
    confirmed: bool


@register_tool
async def add_interests(topics: List[str], ctx: Context = None) -> str:
    """
    Add topics to your interests with confirmation.

    Args:
        topics: List of topics to add

    Returns:
        Confirmation with updated count
    """
    # Ask for confirmation via elicitation
    topics_list = ", ".join(topics)
    result = await ctx.elicit(
        message=f"Are you sure you want to add these {len(topics)} topic(s) to your interests?\n\nTopics: {topics_list}",
        response_type=Confirmation,
    )

    # Handle user response
    if result.action == "decline":
        return "❌ **Operation declined** - No topics were added"
    elif result.action == "cancel":
        return "❌ **Operation cancelled** - No topics were added"
    elif result.action != "accept":
        return "❌ **Invalid response** - No topics were added"

    # Check if user explicitly confirmed
    if not result.data.confirmed:
        return "❌ **Not confirmed** - No topics were added"

    # Proceed with adding topics
    interests_service = ctx.request_context.lifespan_context.interests_service
    add_result = interests_service.add_topics(topics)

    if add_result["success"]:
        output = "✅ **Added interests!**\n\n"
        if add_result["added"]:
            output += "**New topics:**\n"
            for topic in add_result["added"]:
                output += f"- {topic}\n"
        else:
            output += "*All topics were already in your interests*\n"
        output += f"\n**Total topics:** {add_result['total_topics']}"
        return output
    else:
        return f"❌ {add_result['error']}"


@register_tool
async def remove_interests(topics: List[str], ctx: Context = None) -> str:
    """
    Remove topics from your interests with confirmation.

    Args:
        topics: List of topics to remove

    Returns:
        Confirmation with remaining count
    """
    # Ask for confirmation via elicitation
    topics_list = ", ".join(topics)
    result = await ctx.elicit(
        message=f"Are you sure you want to remove these {len(topics)} topic(s) from your interests?\n\nTopics: {topics_list}",
        response_type=Confirmation,
    )

    # Handle user response
    if result.action == "decline":
        return "❌ **Operation declined** - No topics were removed"
    elif result.action == "cancel":
        return "❌ **Operation cancelled** - No topics were removed"
    elif result.action != "accept":
        return "❌ **Invalid response** - No topics were removed"

    # Check if user explicitly confirmed
    if not result.data.confirmed:
        return "❌ **Not confirmed** - No topics were removed"

    # Proceed with removing topics
    interests_service = ctx.request_context.lifespan_context.interests_service
    remove_result = interests_service.remove_topics(topics)

    if remove_result["success"]:
        output = "✅ **Removed interests!**\n\n"
        if remove_result["removed"]:
            output += "**Removed topics:**\n"
            for topic in remove_result["removed"]:
                output += f"- {topic}\n"
        else:
            output += "*No topics were found to remove*\n"
        output += f"\n**Remaining topics:** {remove_result['remaining_topics']}"
        return output
    else:
        return f"❌ {remove_result['error']}"

<div align="center">

<img src="https://github.com/adityaarunsinghal/agentic-ai-workshop-2025/blob/main/notebooks/media/mcp_restaurant_07_elicitation.png?raw=true" width="500" alt="mcp_restaurant_07_elicitation">

### Kitchen wants to know-- "How would you like that cooked?"

</div>

## Checkpoint 4: Elicitation

In [21]:
# warnings.filterwarnings("ignore", category=DeprecationWarning)
# apply_registered_tools(mcp)
# os.system("lsof -ti:8080 | xargs kill -9 2>/dev/null")
# await mcp.run_async(transport="streamable-http", port=8080)

<div align="center">

## Checkpoint 4 Results

<img src="https://github.com/adityaarunsinghal/agentic-ai-workshop-2025/blob/main/notebooks/media/session02checkpoint04_elicitation.png?raw=true" width="1000" alt="session02checkpoint04_elicitation">
<!-- <img src="media/session02checkpoint04_elicitation.png" width="1000" alt="session02checkpoint04_elicitation"> -->

_Much better experience where I control what to add to interests_

_Still extremely high latency and token use_


</div>

---

# The Context Window Problem 💥

## What Happens When We Want Everything?

### The Math of Context Overflow:

- 20 articles minimum
- Each article ≈ 2,000 tokens
- Total: 40,000+ tokens just for content
- Plus: Tool calls, responses, formatting
- **Result: 50,000+ tokens needed!**

### Model Context Limits:

- GPT-4: 128K tokens
- Claude: 200K tokens
- Llama: 32K tokens

---

```bash

Maximum output tokens reached - generation stopped.

─| agent_server |───────────────────────────────────────────────


Last turn: 196,161 Input, 2,048 Output
```

---

And wait...

### The Real Problem:

1. **Cost**: 100K tokens costs $3-5 per request!
2. **Speed**: Large contexts = slow processing
3. **Quality**: LLMs get "lost" in huge contexts
4. **Scaling**: What about 100 interests? 1000 articles?

### This is one of the top challenges we start to face when building Production Agents


In [22]:
# ============= CONTENT ANALYSIS TOOLS (Using Sampling) =============


@register_tool
async def summarize_content(
    content: str, style: str = "balanced", focus: str = "", ctx: Context = None
) -> str:
    """
    Summarize content using LLM sampling.

    Args:
        content: Content to summarize
        style: Summary style (brief, balanced, detailed, technical)
        focus: Optional focus area

    Returns:
        Generated summary
    """
    valid_styles = ["brief", "balanced", "detailed", "technical"]
    if style not in valid_styles:
        return f"❌ Invalid style. Must be one of: {', '.join(valid_styles)}"

    # Style-specific prompts
    style_prompts = {
        "brief": "Summarize in 2-3 sentences, main points only.",
        "balanced": "Create a comprehensive 4-6 sentence summary covering key points.",
        "detailed": "Provide a thorough summary with context, implications, and key details.",
        "technical": "Focus on technical details, methodology, and implementation specifics.",
    }

    prompt = f"""{style_prompts[style]}

{f"Focus particularly on: {focus}" if focus else ""}

Content:
{content[:32000]}

Summary:"""

    try:
        result = await ctx.sample(messages=prompt, temperature=0.3)
        summary = result.text if hasattr(result, "text") else str(result)

        return f"""# Summary ({style} style)

{summary}

*Generated using LLM sampling*"""

    except Exception as e:
        return f"❌ Summarization failed: {e}"


@register_tool
async def extract_key_themes(contents: List[str], ctx: Context = None) -> str:
    """
    Analyze multiple articles and extract common themes.

    Args:
        contents: List of article contents

    Returns:
        Theme analysis
    """
    if not contents:
        return "❌ No contents provided"

    # Combine contents (limit each)
    combined = "\n\n---\n\n".join([c[:1000] for c in contents[:10]])

    prompt = f"""Analyze these {len(contents)} articles and identify:
1. Common themes and trends
2. Connections between stories
3. Emerging patterns
4. Key topics covered

Articles:
{combined}

Analysis:"""

    try:
        result = await ctx.sample(messages=prompt, temperature=0.3)
        analysis = result.text if hasattr(result, "text") else str(result)

        return f"""# Theme Analysis Across {len(contents)} Articles

{analysis}

*Generated using LLM sampling*"""

    except Exception as e:
        return f"❌ Theme extraction failed: {e}"


@register_tool
async def generate_editorial(
    articles: List[Dict], tone: str = "analytical", ctx: Context = None
) -> str:
    """
    Generate an editorial/synthesis piece from multiple articles.

    Args:
        articles: List of article dicts with title and content
        tone: Editorial tone (analytical, educational, skeptical, enthusiastic)

    Returns:
        Generated editorial
    """
    valid_tones = ["analytical", "educational", "skeptical", "enthusiastic"]
    if tone not in valid_tones:
        return f"❌ Invalid tone. Must be one of: {', '.join(valid_tones)}"

    if not articles:
        return "❌ No articles provided"

    # Format articles for prompt
    articles_text = ""
    for i, article in enumerate(articles[:5], 1):
        articles_text += f"\nArticle {i}: {article.get('title', 'Untitled')}\n"
        articles_text += f"{article.get('content', '')[:800]}\n"

    tone_instructions = {
        "analytical": "Provide objective analysis with insights and implications",
        "educational": "Explain concepts clearly and provide context for learning",
        "skeptical": "Question assumptions and highlight potential concerns",
        "enthusiastic": "Highlight exciting developments and positive implications",
    }

    prompt = f"""Write an editorial synthesis piece that connects these stories.

Tone: {tone} - {tone_instructions[tone]}

{articles_text}

Editorial (300-400 words):"""

    try:
        result = await ctx.sample(messages=prompt, temperature=0.5)
        editorial = result.text if hasattr(result, "text") else str(result)

        return f"""# Editorial: {tone.title()} Perspective

{editorial}

*Generated editorial connecting {len(articles)} stories*"""

    except Exception as e:
        return f"❌ Editorial generation failed: {e}"

<div align="center">

<img src="https://github.com/adityaarunsinghal/agentic-ai-workshop-2025/blob/main/notebooks/media/mcp_restaurant_13_llm_reasoning.png?raw=true" width="500" alt="mcp_restaurant_13_llm_reasoning">

<br/>

<img src="https://github.com/adityaarunsinghal/agentic-ai-workshop-2025/blob/main/notebooks/media/mcp_restaurant_08_sampling.png?raw=true" width="500" alt="mcp_restaurant_08_sampling">

### Kitchen asks Chef Miguel for help with something that needs his expertise ("brains")

</div>

In [23]:
# ============= RESOURCES =============

from mcp import Resource

# OpenAI just announced yesterday that exposing HTML like this in resources can enable "ChatGPT Apps" in a few minutes from existing MCP servers
@mcp.resource("memory://latest-newspaper-preview")
async def get_latest_newspaper_preview(ctx: Context = None) -> Resource:
    """Full HTML preview of the latest newspaper."""
    article_memory = ctx.request_context.lifespan_context.article_memory
    newspaper_service = ctx.request_context.lifespan_context.newspaper_service
    email_service = ctx.request_context.lifespan_context.email_service
    
    latest = article_memory.get_latest_newspaper()
    if not latest:
        return Resource(
            uri="memory://latest-newspaper-preview",
            name="Latest Newspaper Preview",
            mimeType="text/html",
            text="<p>No newspapers found</p>"
        )
    
    # Load full newspaper data
    newspaper_data = newspaper_service.get_newspaper_data(latest['newspaper_id'])
    if not newspaper_data:
        return Resource(
            uri="memory://latest-newspaper-preview",
            name="Latest Newspaper Preview",
            mimeType="text/html",
            text="<p>Could not load newspaper data</p>"
        )
    
    # Generate full HTML using your template
    html_content = email_service._create_html_version(newspaper_data, version=2)
    
    return Resource(
        uri="memory://latest-newspaper-preview",
        name="Latest Newspaper Preview",
        mimeType="text/html",
        text=html_content
    )


@mcp.resource("memory://past-newspapers")
async def get_past_newspapers_resource(ctx: Context = None) -> str:
    """Past week's newspapers for context."""
    article_memory = ctx.request_context.lifespan_context.article_memory

    newspapers = article_memory.search_newspapers(days_back=7)

    if not newspapers:
        return "# No Recent Newspapers\n\nNo newspapers found in the past week."

    content = f"# Past Week's Newspapers ({len(newspapers)})\n\n"

    for paper in newspapers:
        content += f"## {paper['title']}\n"
        content += f"- **Date:** {paper['timestamp'][:10]}\n"
        content += f"- **Type:** {paper['edition_type']}\n"
        content += f"- **Articles:** {paper['article_count']}\n"
        if paper["topics"]:
            content += f"- **Topics:** {', '.join(paper['topics'])}\n"
        content += "\n"

    return content


@mcp.resource("memory://article-archive-stats")
async def get_archive_stats(ctx: Context = None) -> str:
    """Statistics about stored articles - topics, sources, etc."""
    article_memory = ctx.request_context.lifespan_context.article_memory

    stats = article_memory.get_stats()

    if "error" in stats:
        return f"# Archive Statistics\n\nError: {stats['error']}"

    content = "# Article Archive Statistics\n\n"

    article_stats = stats["article_archive"]
    content += f"**Total Articles:** {article_stats['total']}\n\n"

    if article_stats["sources"]:
        content += "## Sources\n"
        for source, count in article_stats["sources"].items():
            content += f"- {source}: {count}\n"
        content += "\n"

    if article_stats["top_topics"]:
        content += "## Top Topics\n"
        for topic, count in article_stats["top_topics"].items():
            content += f"- {topic}: {count}\n"
        content += "\n"

    newspaper_stats = stats["newspaper_archive"]
    content += f"**Total Newspapers:** {newspaper_stats['total']}\n\n"

    limits = stats["limits"]
    content += f"**Storage Limits:** {limits['max_items_per_collection']} items, "
    content += f"{limits['max_age_days']} days retention\n"

    return content

In [24]:
# ============= PROMPTS =============


@mcp.prompt()
async def create_deep_dive() -> str:
    """Workflow for creating a comprehensive deep dive newspaper."""
    return """Create a comprehensive deep dive newspaper:

1. Read interests.md and memory://past-newspapers
2. fetch_hn_stories(count=30, category="top")
3. Identify top 3-4 topics matching interests
4. For each topic:
   - Fetch multiple articles
   - search_article_archive() for past coverage
   - Use brave search for additional sources
   - Use perplexity for background research
5. create_newspaper_draft(title="Deep Dive", edition_type="deep_dive")
6. Create sections for each major topic
7. For each article:
   - fetch_article_content()
   - store_article() with topic tags
   - summarize_content(style="detailed", focus=<topic>)
   - add_article() with rich formatting:
     * set_article_format() with key_points, pull_quote
     * Add images if available
   - add_resource_box() with related links
8. Connect related articles with link_related_articles()
9. extract_key_themes() across all content
10. generate_editorial(tone="analytical") for each topic
11. add_theme_highlight() connecting cross-topic stories
12. add_table_of_contents(style="detailed")
13. calculate_reading_times()
14. preview_newspaper(format="markdown") - review structure
15. validate_newspaper() - check for issues
16. send_newspaper()
17. store_newspaper()

Aim for depth and comprehensive coverage - 30-45 min read!"""

@mcp.prompt()
async def interactive_research() -> str:
    """Workflow for interactive deep research session."""
    return """Interactive research session:

1. ELICIT: "What topic interests you today?"
2. search_article_archive(query=<topic>) - check past coverage
3. fetch_hn_stories() - find current discussions
4. Use brave search for <topic>
5. Use perplexity for deep research
6. ELICIT: "Found X sources. Focus on: [technical details] [use cases] [comparisons]?"
7. Based on selection:
   - Fetch relevant articles
   - store_article() for each
   - summarize_content() with appropriate style
8. ELICIT: "Compare to similar technologies?"
9. If yes:
   - Research comparison topics
   - Create comparison tables
10. create_newspaper_draft(title="Research: <topic>", edition_type="special")
11. Organize by user's focus areas
12. Add rich formatting:
    - Comparison tables
    - Key points boxes
    - Resource links
13. extract_key_themes() across all research
14. generate_editorial(tone="educational")
15. ELICIT: "Add more sources or ready to send?"
16. Iterate if needed
17. send_newspaper()
18. store_newspaper()

Highly interactive with multiple decision points!"""

## Checkpoint 5: All CRUD-type tools and extremely high Agentic control over final newspaper

In [ ]:
warnings.filterwarnings("ignore", category=DeprecationWarning)

print("\n" + "=" * 60)
print("🚀 STARTING ADVANCED NEWSPAPER AGENT SERVER")
print("=" * 60)
print("\nAvailable Tools:")
print("  📰 Content Discovery: fetch_hn_stories, fetch_article_content")
print(
    "  💾 Memory: store_article, search_article_archive, store_newspaper, search_past_newspapers"
)
print("  📝 Structure: create_newspaper_draft, add_section, reorder_sections, etc.")
print("  ✨ Articles: add_article, set_article_format, highlight_article, etc.")
print("  🎨 Editorial: add_editors_note, add_theme_highlight, add_stats_callout, etc.")
print(
    "  🔧 Polish: preview_newspaper, validate_newspaper, calculate_reading_times, etc."
)
print("  📧 Delivery: send_newspaper")
print("  🏷️ Interests: read_interests, add_interests, remove_interests")
print("  🤖 Analysis: summarize_content, extract_key_themes, generate_editorial")
print("\nResources:")
print("  📄 file://interests.md")
print("  📰 memory://latest-newspaper")
print("  📚 memory://past-newspapers")
print("  📊 memory://article-archive-stats")
print("\nPrompts:")
print("  ☀️ create_morning_brief")
print("  🔍 create_deep_dive")
print("  💡 interactive_research")
print("\n" + "=" * 60)
print()

apply_registered_tools(mcp)
os.system("lsof -ti:8080 | xargs kill -9 2>/dev/null")
await mcp.run_async(transport="streamable-http", port=8080)
os.system("lsof -ti:8080 | xargs kill -9 2>/dev/null")
await mcp.run_async(transport="streamable-http", port=8080)

<div align="center">

## Our Server is pretty snazzy now and can give meaningful results too...

### But our model is still doing EVERYTHING, and passing lots of tokens back and forth

#### Even the "sampling" so to speak is happening in a way where the model needs to PROVIDE the text which would then be "summarized"

</div>